# Creating a hate speech classifier using PEFT

In [1]:
!pip install -q peft==0.10.0
!pip install -q datasets==2.18.0
!pip install -q accelerate==0.29.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

We'll use the smalleset bloom model

In [3]:
model_name = "bigscience/bloomz-560m"
#model_name = "bigscience/bloomz-1b1"
NUM_VIRTUAL_TOKENS = 20
#If you just want to test the solution, you can reduce the EPOCHs.
NUM_EPOCHS_PROMPT = 10 # 50 -> 10
NUM_EPOCHS_CLASSIFIER = 10
device = "cuda"
# device = "mps" # for Apple Silicon.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map = device
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

# Inference with the Pre Trained Model

In [5]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

In [6]:

input_classifier = tokenizer("Sentence : Head is the shape of a light bulb. Label : ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_classifier.to(device),
                                          max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['Sentence : Head is the shape of a light bulb. Label :  head']


Clearly, the model doesn't have any idea what is happening.

# Hate Speech Dataset

In [7]:
import os
from datasets import load_dataset

In [8]:
dataset_classifier = "SetFit/ethos_binary"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [9]:
data_classifier = load_dataset(dataset_classifier)
data_classifier['train'] = concatenate_columns_classifier(data_classifier['train'])

data_classifier = data_classifier.map(lambda samples: tokenizer(samples["text"]), batched=True)
#  Remove all columns in the dataset that are not relavant for training the model
train_sample_classifier = data_classifier["train"].remove_columns(['label', 'label_text', 'text'])

Repo card metadata block was not found. Setting CardData to empty.


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [10]:
data_classifier


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 598
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [11]:
train_sample_classifier

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 598
})

In [12]:
print(train_sample_classifier[1:2])

{'input_ids': [[62121, 1671, 915, 473, 760, 10190, 513, 16154, 60, 19821, 138929, 20812, 426, 18833, 18816, 75536, 45617, 39469, 19368, 17956, 57274, 3758, 18065, 38, 44140, 17956, 72870, 8309, 9492, 15, 614, 156801, 85061, 48283, 44419, 426, 16472, 96789, 602, 45227, 43111, 181485, 435, 19821, 60, 48283, 44419, 426, 16472, 96789, 614, 156801, 77658, 915, 74549, 40423]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


# Prompt Tuning

In [13]:
from peft import PromptTuningInit, PromptTuningConfig, TaskType, get_peft_model

In [14]:
generation_config_classifier = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.TEXT,  #
    prompt_tuning_init_text="Indicates whether the sentence contains hate speech or not",
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)

In [15]:
peft_model_classifier = get_peft_model(foundational_model, generation_config_classifier)
print(peft_model_classifier.print_trainable_parameters())

trainable params: 20,480 || all params: 559,235,072 || trainable%: 0.0036621451381361144
None


In [16]:
import os

working_dir = "./"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
# output_directory_prompt =  os.path.join(working_dir, "peft_outputs_prompt")
output_directory_classifier =  os.path.join(working_dir, "peft_outputs_classifier")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_directory_classifier):
    os.mkdir(output_directory_classifier)

In [17]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.0035, epochs=6, autobatch=True):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        #use_cpu=True, # This is necessary for CPU clusters.
        auto_find_batch_size=autobatch, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full fine-tuning
        #per_device_train_batch_size=4,
        num_train_epochs=epochs
    )
    return training_args

In [18]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        # We pass in the PEFT version of the foundation model, bloomz-560M
        model=model,
        #The args for the training.
        args=training_args,
        train_dataset=train_dataset, #The dataset used to train the model.
        # mlm=False indicates not to use masked language modeling
        data_collator=DataCollatorForLanguageModeling(
            tokenizer,
            mlm=False)
    )
    return trainer

In [19]:
training_args_classifier = create_training_arguments(output_directory_classifier,
                                                    3e-2,
                                                    NUM_EPOCHS_CLASSIFIER)

In [20]:
trainer_classifier = create_trainer(peft_model_classifier,
                                   training_args_classifier,
                                   train_sample_classifier)
trainer_classifier.train()

Step,Training Loss


Step,Training Loss


Step,Training Loss


Step,Training Loss
500,3.254100
1000,3.186500
1500,3.186700
2000,3.161800
2500,3.139000
3000,3.140200
3500,3.092100
4000,3.096200
4500,3.121400
5000,3.068900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=5980, training_loss=3.129223612399404, metrics={'train_runtime': 519.6848, 'train_samples_per_second': 11.507, 'train_steps_per_second': 11.507, 'total_flos': 555878292013056.0, 'train_loss': 3.129223612399404, 'epoch': 10.0})

In [21]:
# Save the model
trainer_classifier.model.save_pretrained(output_directory_classifier)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Inference from our model

In [25]:
from peft import PeftModel

loaded_model_peft = PeftModel.from_pretrained(foundational_model,
                                         output_directory_classifier,
                                         #device_map=device,
                                         is_trainable=False)

In [26]:
loaded_model_peft.load_adapter(output_directory_classifier, adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [27]:
loaded_model_sentences_outputs = get_outputs(loaded_model_peft,
                                             input_classifier, max_new_tokens=3)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens=True))

['Sentence : Head is the shape of a light bulb. Label :  no hate speech']


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
